In [49]:
# Ataque de Coster al criptosistema de Merkle-Hellman
# Juan Manuel Mateos Pérez

In [50]:
## Explicación :
# En este programa estamos simulando el ataque mediante el método de Coster a una comunicación realizada usando el 
# criptosistema de Merkle-Hellman. El ataque utiliza únicamente los valores conocidos del criptosistema, que son la
# clave pública y el mensaje cifrado. El programa obtiene como resultado un mensaje descifrado, el cual comprobaremos
# si coincide con el original.

## Ejecución :
# Para ejecutar el programa, solo debemos descomentar el código del main que queramos utilizar:
# (1) Si descomentamos la primera parte, podremos ejecutar el programa 1 vez y veremos todos los datos necesarios. Además, 
# podemos modificar el tamaño del mensaje (variable tam) y el número de iteraciones de la clave privada (variable it).
# (2) Si por otro lado descomentamos la segunda parte, el programa se ejecuta p veces y mostrará un desglose de fallos, 
# vacios y errores de longitud cometidos. En este caso podemos modificar la variable p, que indica la cantidad de cripto-
# sistemas que se van a ejecutar.
# (3) Finalmente, si descomentamos la tercera parte, el programa ejecuta una función que mide el tiempo medio de ejecución
# tras ejecutar el criptosistema p veces. Aquí, se puede modificar el tamaño del mensaje (variable tam), el número de 
# iteraciones de la clave privada (variable it) y el número de ejecuciones totales (variable p).

In [51]:
# Importamos el método de Merkle-Hellman
load("Merkle-Hellman_iterativo.py")

In [52]:
# ATAQUE DE COSTER

In [53]:
# Genera la matriz necesaria para aplicar Coster
def generarMatriz(pk, s):
    n = len(pk)
    N = random.randint(int((1/2)*sqrt(n)), int(sqrt(n)))
    filas = []
    
    # generamos los n primeros vectores
    for i in range(0, n):
        aux = zero_vector(n+1)
        aux[i] = 1
        aux[n] = pk[i]*N
        filas.append(aux)

    # generamos el vector n+1
    b = []
    for i in range(n+1):
        b.append(1/2)
    b[n] = s*N
        
    filas.append(b)
    
    return Matrix(filas)

In [54]:
# Encuentra una solución en la matriz recibida
def buscarSolucion(matriz, pk, s):
    n = matriz.nrows()
    sol_encontrada = False
    solucion = []
    
    for j in range(n):
        if sol_encontrada == False:
            suma = 0
            fila = matriz[j]
        
            for i in range(len(fila)-1):
                aux = fila[i] + (1/2)
                if aux < 0:
                    aux = 0
                suma += pk[i]*aux
                solucion.append(aux)
            if suma == s:
                sol_encontrada = True
            else:
                solucion = []

    solucion = list(solucion)
    return solucion

In [55]:
# Intercambia los 1/2 por -1/2 y viceversa en una matriz
def cambiarMatriz(matriz):
    matriz_res = copy(matriz)

    for i in range(matriz.nrows()):
        for j in range(matriz.ncols()-1):
            if matriz[i, j] == 1/2:
                matriz_res[i, j] = -1/2
            elif matriz[i, j] == -1/2:
                matriz_res[i, j] = 1/2
    
    return matriz_res

In [56]:
# Aplica el ataque de Coster
def ataqueCoster(pk, s):
    n = len(pk)
    encontrado = False
    solucion = []
    
    # generamos la matriz
    matriz_ini = generarMatriz(pk, s)
    # aplicamos LLL
    matriz_res = matriz_ini.LLL()
    # buscamos una solución
    solucion = buscarSolucion(matriz_res, pk, s)
    
    # cambiamos los 1/2 por -1/2 y viceversa
    if len(solucion) == 0:
        matriz_cambio = cambiarMatriz(matriz_res)
        solucion = buscarSolucion(matriz_cambio, pk, s)
        
    return solucion

In [57]:
# Calcula el número de errores cometidos
def comprobarErrores(men_orig, men_obt):
    n = len(men_orig)
    vector_dif = []

    if len(men_obt) == 0:
        return len(men_orig)

    for i in range(0, n):
        vector_dif.append(abs(men_orig[i] - men_obt[i]))

    return sum(vector_dif)

In [58]:
# DATOS DE SALIDA

In [59]:
# ejecuta y muestra los datos tras aplicar una iteración
def unaIteracion(tam, it):
    merkle_hellman = Merkle_Hellman(tam, it)
    merkle_hellman.do()

    pk = merkle_hellman.pk
    s  = merkle_hellman.s

    coster = ataqueCoster(pk, s)

    print()
    print("Clave pública      :", merkle_hellman.pk)
    print("Mensaje original   :", merkle_hellman.mensaje)
    print("Mensaje cifrado    :", merkle_hellman.s)
    print("Mensaje descifrado :", coster)
    print("Tamaño mensaje     :", tam)
    print("Número iteraciones :", it)
    print("Errores totales    :", comprobarErrores(merkle_hellman.mensaje, coster))

In [60]:
# ejecuta y muestra los datos tras aplicar n iteraciones
def variasIteraciones(n):
    errores  = 0
    err_long = 0
    vacios   = 0
    densidad = 0

    print("Iteración \t Tamaño vector \t Número Iteraciones \t Densidad \t\tResultado")
    for i in range(p):
        print(i+1, end = "")

        tam = random.randint(3, 100)
        print("\t\t", tam, end="")

        it = random.randint(0, 3)
        print("\t\t", it, end="")

        merkle_hellman = Merkle_Hellman(tam, it)
        merkle_hellman.do()

        densidad = RR(merkle_hellman.tamano / log(max(merkle_hellman.pk), 2))
        print("\t\t\t", densidad, end="")

        coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
        if len(coster) == 0:
            vacios += 1
            print("\tvacio")
        elif len(coster) != len(merkle_hellman.mensaje):
            err_long += 1
            print("\terror longitud")
        else:
            valor = comprobarErrores(merkle_hellman.mensaje, coster)
            errores += valor
            if valor != 0:
                print("\terror valor")
            else:
                print("\tobtenido")

    print("\nErrores totales  tras", p, "iteraciones :", errores)
    print("Vacios  totales  tras", p, "iteraciones :", vacios)
    print("Errores longitud tras", p, "iteraciones :", err_long)

In [61]:
# ejecuta y mide los tiempos tras aplicar n iteraciones
def medirTiempos(tam, it, p):
    tiempo_medio = 0
    vacios = 0
    for i in range(p):
        merkle_hellman = Merkle_Hellman(tam, it)
        merkle_hellman.do()

        inicio = time.time()
        coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
        fin = time.time()
        tiempo_medio += fin - inicio

        if len(coster) == 0:
            vacios += 1

    tiempo_medio /= p
    print("Tamaño          :", tam)
    print("Iteraciones     :", it)
    print("Ejecuciones     :", p)
    print("Tiempo medio    :", tiempo_medio)
    print("Errores totales :", vacios)

In [62]:
def medirTodo(tam, p):
    tiempo_medio1 = 0
    tiempo_medio2 = 0
    vacios1 = 0
    vacios2 = 0
    
    for i in range(p):
        # con 0 iteraciones de sk
        merkle_hellman = Merkle_Hellman(tam, 0)
        merkle_hellman.do()

        inicio = time.time()
        coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
        fin = time.time()
        tiempo_medio1 += fin - inicio

        if len(coster) == 0:
            vacios1 += 1
        
        # con 3 iteraciones de sk
        merkle_hellman = Merkle_Hellman(tam, 3, merkle_hellman.mensaje, merkle_hellman.sk[0])
        merkle_hellman.do()
        
        inicio = time.time()
        coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
        fin = time.time()
        tiempo_medio2 += fin - inicio
        
        if len(coster) == 0:
            vacios2 += 1

    tiempo_medio1 /= p
    tiempo_medio2 /= p
    print("Tamaño          :", tam)
    print("Iteraciones     : 0")
    print("Ejecuciones     :", p)
    print("Tiempo medio    :", tiempo_medio1)
    print("Errores totales :", vacios1)
    print("Tamaño          :", tam)
    print("Iteraciones     : 3")
    print("Ejecuciones     :", p)
    print("Tiempo medio    :", tiempo_medio2)
    print("Errores totales :", vacios2
    print()    

In [63]:
# MAIN

In [64]:
import time
print("Ataque Coster")

# ---------- descomentar para realizar 1 ejecución aleatoria ----------
# tam = random.randint(3, 100)
# it  = random.randint(0, 3)
# unaIteracion(tam, it)

# ---------- descomentar para realizar p ejecuciones aleatorias ----------
# p = 10
# variasIteraciones(p)

# ---------- descomentar para medir tiempos de p ejecuciones aleatorias ----------
# print()
# tam = 50
# it  = 3
# p   = 100
# medirTiempos(tam, it, p)

# print()
# tam = 75
# it  = 3
# p   = 100
# medirTiempos(tam, it, p)

# print()
# tam = 100
# it  = 3
# p   = 100
# medirTiempos(tam, it, p)

# print()
# tam = 50
# it  = 0
# p   = 100
# medirTiempos(tam, it, p)

# print()
# tam = 75
# it  = 0
# p   = 100
# medirTiempos(tam, it, p)

# print()
# tam = 100
# it  = 0
# p   = 100
# medirTiempos(tam, it, p)

medirTodo(5, 20)
medirTodo(25, 20)
medirTodo(50, 20)
medirTodo(75, 20)
medirTodo(100, 20)

print("-----------------------------------------------------------------------------------------------------------------")

medirTodo(5, 100)
medirTodo(25, 100)
medirTodo(50, 100)
medirTodo(75, 100)
medirTodo(100, 100)

Ataque Coster
Tamaño          : 5
Iteraciones     : 0
Ejecuciones     : 10
Tiempo medio    : 0.026523828506469727
Errores totales : 0
Tamaño          : 5
Iteraciones     : 3
Ejecuciones     : 10
Tiempo medio    : 0.026170969009399414
Errores totales : 0
